# Demo of Survival Analysis Package

## Step 1: Database Schema Initiation

In [1]:
from survival_analysis.database_preparation.schema import *

2023-12-01 11:57:20,659 - schema.py - <module> - INFO - (Schema Has Been Created) - line: 142


## Step 2: Adding Customer Data to the DB

In [6]:
from survival_analysis.database_preparation.sql_interactions\
    import SqlHandler
import pandas as pd

* **Getting Data from a CSV File**

In [3]:
# For Windows users
data = pd.read_csv(r'Raw Data\telco.csv')

# For Mac Users
#data = pd.read_csv(r'Raw Data/telco.csv')
data.head()

,Customer_ID,region,tenure,Age,Marital_Status,Address_ID,income,Education,Retirement,gender,Voice_Included,Internet_Included,Forward_Included,service_category,churn
0,1,Zone 2,13,44,Married,9,64,College degree,No,Male,No,No,Yes,Basic service,Yes
1,2,Zone 3,11,33,Married,7,136,Post-undergraduate degree,No,Male,Yes,No,Yes,Total service,Yes
2,3,Zone 3,68,52,Married,24,116,Did not complete high school,No,Female,No,No,No,Plus service,No
3,4,Zone 2,33,33,Unmarried,12,33,High school degree,No,Female,No,No,No,Basic service,Yes
4,5,Zone 2,23,30,Married,9,30,Did not complete high school,No,Male,No,No,Yes,Plus service,No


* **Populating the 'DimCustomer' table**

In [4]:
Inst = SqlHandler('sa_db', 'DimCustomer')
Inst.insert_many(data) 
Inst.close_cnxn()

2023-12-01 12:02:09,450 - sql_interactions.py - insert_many - INFO - (BEFORE the column intersection: ['customer_id', 'region', 'tenure', 'age', 'marital_status', 'address_id', 'income', 'education', 'retirement', 'gender', 'voice_included', 'internet_included', 'forward_included', 'service_category', 'churn']) - line: 173
2023-12-01 12:02:09,454 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['Customer_ID', 'Age', 'Tenure', 'Gender', 'Income', 'Marital_Status', 'Address_ID', 'Education', 'Retirement', 'Churn', 'Region', 'Service_Category', 'Voice_Included', 'Internet_Included', 'Forward_Included']) - line: 136
2023-12-01 12:02:09,456 - sql_interactions.py - insert_many - INFO - (AFTER the column intersection: ['address_id', 'age', 'tenure', 'customer_id', 'marital_status', 'income', 'education', 'churn', 'internet_included', 'voice_included', 'retirement', 'forward_included', 'service_category', 'region', 'gender']) - line: 176
2023-12-01 12:02:09,474 - sql_i

## Step 3: Running the Model to Estimate Customer Churn Rate and CLV

In [5]:
from survival_analysis.model_preparation.model_AFT import AFTModelSelector
from survival_analysis.database_preparation.sql_interactions\
    import SqlHandler
from survival_analysis.utils import format_dataframe
import pandas as pd

* **Getting Data From the DB**

In [2]:
#Initiate the connection
Inst=SqlHandler('sa_db', 'DimCustomer')

#Get data in dataframe format
df = Inst.from_sql_to_pandas(chunksize=100, id_value = 'Customer_ID')

#Close the connection
Inst.close_cnxn()

2023-12-01 21:56:01,565 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,584 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,599 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,614 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,634 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,654 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,670 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-01 21:56:01,688 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-

* **Dumifying the Categorical Variables**

In [5]:
df = format_dataframe(df)

* **Running the Model**
1. Chooses the Best Accelerated Failure Time (AFT) Model Based on AIC. 
2. Computes Churn Rates & CLV For Each Customer for Specified Number of Periods. 
3. Returns the Results in a Dataframe. 

In [8]:
#Running the AFT model for next 12 time periods
duration_column = 'Tenure' 
event_column = 'Churn_Yes'
primary = 'Customer_ID'

aft_model_selector = AFTModelSelector(df, primary,\
                    duration_column, event_column)
aft_model_selector.select_best_model()
aft_model_selector.fit_and_predict(n_time_periods=12)
aft_model_selector.calculate_clv()
aft_model_selector.predictions_df.head()

2023-12-01 22:00:05,411 - model_AFT.py - select_best_model - INFO - (Weibull AIC: 2958.357126946646) - line: 86
2023-12-01 22:00:06,096 - model_AFT.py - select_best_model - INFO - (Exponential AIC: 3052.729101281061) - line: 86
2023-12-01 22:00:07,905 - model_AFT.py - select_best_model - INFO - (LogNormal AIC: 2950.3002131922785) - line: 86
2023-12-01 22:00:09,421 - model_AFT.py - select_best_model - INFO - (LogLogistic AIC: 2952.2582596356688) - line: 86
2023-12-01 22:00:09,423 - model_AFT.py - select_best_model - WARNING - (
Best Model: LogNormal with AIC: 2950.3002131922785) - line: 92
2023-12-01 22:00:09,706 - model_AFT.py - fit_and_predict - INFO - (The AFT model was run successfully.) - line: 138
2023-12-01 22:00:09,914 - model_AFT.py - calculate_clv - INFO - (The CLV predictions were added successfully.) - line: 188


,customer_id,pred_period,churn_rate,CLV
0,1,1,0.00060,1299.220
1,2,1,0.00013,1299.831
2,3,1,0.00000,1300.000
3,4,1,0.00115,1298.505
4,5,1,0.00006,1299.922


* **Saving Results to a CSV File**

In [9]:
#For Windows
aft_model_selector.predictions_df.to_csv\
    ('Raw Data\model_output.csv', index=False)

#For MAC
#aft_model_selector.predictions_df.to_csv\
# ('Raw Data/model_output.csv', index=False)

* **Populating 'FactPredictions' Table**

In [11]:
#For Windows
data =pd.read_csv(r'Raw Data\model_output.csv')

#For MAC
#data =pd.read_csv(r'Raw Data/model_output.csv')

#Make sure table is initially empty
Inst=SqlHandler('sa_db', 'FactPredictions') #initiate connection
Inst.truncate_table() #make sure table is initially empty
Inst.close_cnxn() #Close the connection

#Inserting the results to DB 
Inst=SqlHandler('sa_db', 'FactPredictions') #initiate connection
Inst.insert_many(data)
Inst.close_cnxn() #Close the connection

2023-12-01 22:05:05,284 - sql_interactions.py - close_cnxn - INFO - (Committing the changes) - line: 77
INFO:sql_interactions.py:Committing the changes
2023-12-01 22:05:05,293 - sql_interactions.py - close_cnxn - INFO - (The connection has been closed) - line: 79
INFO:sql_interactions.py:The connection has been closed
2023-12-01 22:05:05,308 - sql_interactions.py - insert_many - INFO - (BEFORE the column intersection: ['customer_id', 'pred_period', 'churn_rate', 'clv']) - line: 173
INFO:sql_interactions.py:BEFORE the column intersection: ['customer_id', 'pred_period', 'churn_rate', 'clv']
2023-12-01 22:05:05,319 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['pred_period', 'customer_ID', 'CLV', 'Churn_Rate']) - line: 136
INFO:sql_interactions.py:The list of columns: ['pred_period', 'customer_ID', 'CLV', 'Churn_Rate']
2023-12-01 22:05:05,330 - sql_interactions.py - insert_many - INFO - (AFTER the column intersection: ['pred_period', 'customer_id', 'churn_rate'

# Step 4: Running the API

In [3]:
#Importing libraries
from survival_analysis.api import app
import uvicorn
import os
import fastapi

if __name__== "__main__":
    uvicorn.run(app)

RuntimeError: asyncio.run() cannot be called from a running event loop